In [ ]:
pip install datasets

In [17]:
#pip install transformers==4.29.2
pip install transformers==4.29.2

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: /apps/z_install_tree/linux-rocky8-ivybridge/gcc-12.2.0/ood-jupyterlab-3.4.8-3qopzgrtpuwifxuotyxofkeze52d6ra7/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
#pip install transformers==4.17.0

In [ ]:
pip install jiwer

In [ ]:
pip install IPython

In [ ]:
pip install IPython.display

In [ ]:
pip install torch

In [ ]:
pip install --upgrade accelerate

In [ ]:
pip install git+https://github.com/huggingface/accelerate

In [ ]:
pip install accelerate -U

In [ ]:
# Based on: 
#     https://huggingface.co/blog/fine-tune-wav2vec2-english
#     https://colab.research.google.com/github/patrickvonplaten/notebooks/blob/master/Fine_tuning_Wav2Vec2_for_English_ASR.ipynb#scrollTo=e7cqAWIayn6w
#     https://github.com/monomest/Thesis/blob/3a15f747dfd934535ffb7a02bf3fee97d9c546cb/s5/wav2vec_projects/THESIS_C/run_finetune_kids_OGI.py#L239

In [1]:
print("\n------------ Importing libraries... ------------\n")
import pandas as pd
import random
import json
import librosa
import torch
import numpy as np

import transformers
from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor
from transformers import Wav2Vec2ForCTC
from transformers import TrainingArguments
from transformers import Trainer

from datasets import load_dataset, load_metric, ClassLabel, Audio
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union


------------ Importing libraries... ------------



/apps/z_install_tree/linux-rocky8-ivybridge/gcc-12.2.0/tensorflow-2.11.0-wwrg4vksfz6yhvhh2tlpr6nbv4qezh26/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-06-05 12:55:50.428362: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-05 12:55:50.755860: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023

In [2]:
print("pandas version:", pd.__version__)
print("json version:", json.__version__)
print("librosa version:", librosa.__version__)
print("Numpy version:", np.__version__)
print("Transformers version:", transformers.__version__)
print("Torch version:", torch.__version__)
print("Test cuda_device_count", torch.cuda.device_count())
print("Test cuda_is_available", torch.cuda.is_available())
print("Test get_device_name", torch.cuda.get_device_name(0))

pandas version: 2.0.1
json version: 2.0.9
librosa version: 0.10.0.post2
Numpy version: 1.24.2
Transformers version: 4.29.2
Torch version: 1.13.1+cu117
Test cuda_device_count 1
Test cuda_is_available True
Test get_device_name Tesla V100-SXM2-32GB


In [3]:
print("\n------------------ Model arguments... ------------------\n")
# For setting model = Wav2Vec2ForCTC.from_pretrained()

set_hidden_dropout = 0.1                    # Default = 0.1
print("hidden_dropout:", set_hidden_dropout)
set_activation_dropout = 0.1                # Default = 0.1
print("activation_dropout:", set_activation_dropout)
set_attention_dropout = 0.1                 # Default = 0.1
print("attention_dropout:", set_attention_dropout)
set_feat_proj_dropout = 0.0                 # Default = 0.1
print("feat_proj_dropout:", set_feat_proj_dropout)
set_layerdrop = 0.01                        # Default = 0.1
print("layerdrop:", set_layerdrop)
set_mask_time_prob = 0.065                  # Default = 0.05
print("mask_time_prob:", set_mask_time_prob)
set_mask_time_length = 10                   # Default = 10
print("mask_time_length:", set_mask_time_length)
set_ctc_loss_reduction = "mean"             # Default = "sum"
print("ctc_loss_reduction:", set_ctc_loss_reduction)
set_ctc_zero_infinity = True               # Default = False
print("ctc_zero_infinity:", set_ctc_zero_infinity)



------------------ Model arguments... ------------------

hidden_dropout: 0.1
activation_dropout: 0.1
attention_dropout: 0.1
feat_proj_dropout: 0.0
layerdrop: 0.01
mask_time_prob: 0.065
mask_time_length: 10
ctc_loss_reduction: mean
ctc_zero_infinity: True


In [4]:
print("\n------------------ Training arguments... ------------------\n")
# For setting training_args = TrainingArguments()

set_per_device_train_batch_size = 8         # Default = 8
print("per_device_train_batch_size:", set_per_device_train_batch_size)
set_group_by_length = True                  # Default = False
print("group_by_length:", set_group_by_length)
set_gradient_accumulation_steps = 1         # Default = 1
print("gradient_accumulation_steps:", set_gradient_accumulation_steps)
set_gradient_checkpointing = True           # Default = False
print("gradient_checkpointing:", set_gradient_checkpointing)
set_weight_decay = 0.01                     # Default = 0
print("weight_decay:", set_weight_decay)
set_fp16 = True                             # Default = False
print("fp16:", set_fp16)

set_learning_rate = 0.00004                 # Default = 0.00005
print("learning_rate:", set_learning_rate)
set_lr_scheduler_type = "linear"            # Default = "linear"
print("lr_scheduler_type:", set_lr_scheduler_type )
set_adam_beta1 = 0.9                        # Default = 0.9
print("adam_beta1:", set_adam_beta1)
set_adam_beta2 = 0.98                       # Default = 0.999
print("adam_beta2:", set_adam_beta2)
set_adam_epsilon = 0.00000001               # Default = 0.00000001
print("adam_epsilon:", set_adam_epsilon)
set_warmup_ratio = 0.1                      # Default = 0.0
print("warmup_ratio:", set_warmup_ratio)

set_num_train_epochs = 22                 # Default = 3.0
print("num_train_epochs:", set_num_train_epochs)
set_max_steps = 35000                       # Default = -1, overrides epochs
print("max_steps:", set_max_steps)

set_logging_strategy = "steps"              # Default = "steps"
print("logging_strategy:", set_logging_strategy)
set_logging_steps = 1000                      # Default = 500
print("logging_steps:", set_logging_steps)
set_save_strategy = "steps"                 # Default = "steps"
print("save_strategy:", set_save_strategy)
set_save_steps = 1000                         # Default = 500
print("save_steps:", set_save_steps)
set_evaluation_strategy = "steps"           # Default = "no"
print("evaluation_strategy:", set_evaluation_strategy)
set_eval_steps = 1000                         # Optional
print("eval_steps:", set_eval_steps)
set_save_total_limit = 40                   # Optional                 
print("save_total_limit:", set_save_total_limit)

set_load_best_model_at_end = True           # Default = False
print("load_best_model_at_end:", set_load_best_model_at_end)
set_metric_for_best_model = "wer"           # Optional
print("metric_for_best_model:", set_metric_for_best_model)
set_greater_is_better = False               # Optional
print("greater_is_better:", set_greater_is_better)


------------------ Training arguments... ------------------

per_device_train_batch_size: 8
group_by_length: True
gradient_accumulation_steps: 1
gradient_checkpointing: True
weight_decay: 0.01
fp16: True
learning_rate: 4e-05
lr_scheduler_type: linear
adam_beta1: 0.9
adam_beta2: 0.98
adam_epsilon: 1e-08
warmup_ratio: 0.1
num_train_epochs: 22
max_steps: 35000
logging_strategy: steps
logging_steps: 1000
save_strategy: steps
save_steps: 1000
evaluation_strategy: steps
eval_steps: 1000
save_total_limit: 40
load_best_model_at_end: True
metric_for_best_model: wer
greater_is_better: False


In [5]:
print("\n------------------ Loading files... ------------------\n")

train_df_fp = '/srv/scratch/z5313567/thesis/OGI_local/short_datasets/short_OGI_scripted_train_dataframe.csv'
dev_df_fp = '/srv/scratch/z5313567/thesis/OGI_local/short_datasets/short_OGI_scripted_dev_dataframe.csv'
test_df_fp = '/srv/scratch/z5313567/thesis/OGI_local/short_datasets/short_OGI_scripted_test_dataframe.csv'
cache_fp = '/srv/scratch/chacmod/.cache/huggingface/datasets/Jordan-OGI-finetune'
model_fp = '/srv/scratch/z5313567/thesis/OGI_local/short_datasets/non-offical-OGI'
vocab_fp = '/srv/scratch/z5313567/thesis/OGI_local/short_datasets/vocab_non-offical-OGI.json'

print(f'Training dataset is stored at {train_df_fp}\n')
print(f'Development dataset is stored at {dev_df_fp}\n')
print(f'Testing dataset is stored at {test_df_fp}\n')
print(f'Cache filepath is {cache_fp}\n')
print(f'Model filepath is {model_fp}\n')
print(f'Vocab filepath is {vocab_fp}\n')


------------------ Loading files... ------------------

Training dataset is stored at /srv/scratch/z5313567/thesis/OGI_local/short_datasets/short_OGI_scripted_train_dataframe.csv

Development dataset is stored at /srv/scratch/z5313567/thesis/OGI_local/short_datasets/short_OGI_scripted_dev_dataframe.csv

Testing dataset is stored at /srv/scratch/z5313567/thesis/OGI_local/short_datasets/short_OGI_scripted_test_dataframe.csv

Cache filepath is /srv/scratch/chacmod/.cache/huggingface/datasets/Jordan-OGI-finetune

Model filepath is /srv/scratch/z5313567/thesis/OGI_local/short_datasets/non-offical-OGI

Vocab filepath is /srv/scratch/z5313567/thesis/OGI_local/short_datasets/vocab_non-offical-OGI.json



In [6]:
print("\n------------------ Loading datasets... ------------------\n")

dataset = load_dataset('csv', 
                    data_files={'train': train_df_fp,
                                'dev': dev_df_fp,
                                'test': test_df_fp},
                    cache_dir=cache_fp)


------------------ Loading datasets... ------------------



Found cached dataset csv (/srv/scratch/chacmod/.cache/huggingface/datasets/Jordan-OGI-finetune/csv/default-631b16b4329d1149/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 3/3 [00:00<00:00, 232.47it/s]


In [7]:
#dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [8]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['filepath', 'duration', 'speaker_id', 'transcription'],
        num_rows: 470
    })
    dev: Dataset({
        features: ['filepath', 'duration', 'speaker_id', 'transcription'],
        num_rows: 134
    })
    test: Dataset({
        features: ['filepath', 'duration', 'speaker_id', 'transcription'],
        num_rows: 132
    })
})


In [9]:
print("\n------------------ Showing some random elements... ------------------\n")

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), f"Can't pick more elements than there are in the dataset {len(dataset)}." 
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    print(df)


------------------ Showing some random elements... ------------------



In [10]:
show_random_elements(dataset["train"].remove_columns(["filepath", "duration", "speaker_id"]), num_examples=10)

                           transcription
0                 we had a birdseye view
1    five people can fit in the elevator
2             help me unroll the new rug
3                             pathfinder
4            we've had a wonderful night
5                              civilized
6                              civilized
7                                 ethnic
8  mabel grows tomatoes in the courtyard
9                  i think cats are cute


In [11]:
print("\n------------------ Extracting individual characters... ------------------\n")
def extract_all_chars(batch):
    all_text = " ".join(batch["transcription"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}


------------------ Extracting individual characters... ------------------



In [12]:
vocabs = dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dataset.column_names["train"])

In [13]:
print('vocabs is:', vocabs)

vocabs is: DatasetDict({
    train: Dataset({
        features: ['vocab', 'all_text'],
        num_rows: 1
    })
    dev: Dataset({
        features: ['vocab', 'all_text'],
        num_rows: 1
    })
    test: Dataset({
        features: ['vocab', 'all_text'],
        num_rows: 1
    })
})


In [14]:
#dataset.column_names["train"]

In [15]:
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

In [16]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
print("vocab_dict is:", vocab_dict)

vocab_dict is: {' ': 0, 'i': 1, 'x': 2, "'": 3, 't': 4, 'm': 5, 'r': 6, 'c': 7, 'l': 8, 'v': 9, 'a': 10, 'y': 11, 'o': 12, 'e': 13, 'w': 14, 'p': 15, 'z': 16, 'u': 17, 'f': 18, 's': 19, 'n': 20, 'd': 21, 'j': 22, 'k': 23, 'q': 24, 'b': 25, 'g': 26, 'h': 27}


In [17]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(f'The lendth of vocab_list is {len(vocab_dict)}')

The lendth of vocab_list is 30


In [18]:
with open(vocab_fp, 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)
print('Successfully created vocab.json file at:', vocab_fp)

Successfully created vocab.json file at: /srv/scratch/z5313567/thesis/OGI_local/short_datasets/vocab_non-offical-OGI.json


In [19]:
print("\n------------------ Creating tokenizer... ------------------\n")
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")


------------------ Creating tokenizer... ------------------



In [20]:
print("\n------------------ Creating feature extractor... ------------------\n")
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)


------------------ Creating feature extractor... ------------------



In [21]:
print("\n------------------ Creating processor... ------------------\n")
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)


------------------ Creating processor... ------------------



In [22]:
def process_dataset(batch):
    batch['speech'], batch['sampling_rate'] = librosa.load(batch['filepath'], sr=feature_extractor.sampling_rate)
    batch["target_text"] = batch["transcription"]
    return batch


In [23]:
print("\n------------------ Obtaining speech arrays, sampling rates, and target texts... ------------------\n")
dataset = dataset.map(process_dataset, remove_columns=dataset.column_names["train"], num_proc=4)


------------------ Obtaining speech arrays, sampling rates, and target texts... ------------------



Loading cached processed dataset at /srv/scratch/chacmod/.cache/huggingface/datasets/Jordan-OGI-finetune/csv/default-631b16b4329d1149/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-6c63f1f1b08c919d_*_of_00004.arrow
Loading cached processed dataset at /srv/scratch/chacmod/.cache/huggingface/datasets/Jordan-OGI-finetune/csv/default-631b16b4329d1149/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-5129c85c3add470e_*_of_00004.arrow
Loading cached processed dataset at /srv/scratch/chacmod/.cache/huggingface/datasets/Jordan-OGI-finetune/csv/default-631b16b4329d1149/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-09b14f759d9ab03e_*_of_00004.arrow


In [24]:
print('dataset is:', dataset)

dataset is: DatasetDict({
    train: Dataset({
        features: ['speech', 'sampling_rate', 'target_text'],
        num_rows: 470
    })
    dev: Dataset({
        features: ['speech', 'sampling_rate', 'target_text'],
        num_rows: 134
    })
    test: Dataset({
        features: ['speech', 'sampling_rate', 'target_text'],
        num_rows: 132
    })
})


In [25]:

# the sampling rate of the data that was used to pretrain the model should match the sampling rate of the dataset used to fine-tune the model.
def check_sampling_rate(dataset): # say, dataset = dataset['train']
    target_sr = feature_extractor.sampling_rate
    if len(set(dataset['sampling_rate'])) == 1:
        actual_sr = list(set(dataset['sampling_rate']))[0] # sampling rate = 22050 Hz for OGI corpus
        if actual_sr != target_sr:
            print('MISMATCH!: the sampling rate used for fine-tuning Wav2vec2.0 does not match the sampling rate used for pretraining')
            
            now = datetime.now()
            # Print out dd/mm/YY H:M:S
            # ------------------------------------------
            dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
            print('Resampling starts...', dt_string)
            for i in range(len(dataset)):
                dataset[i]['speech'] = librosa.resample(np.asarray(dataset[i]['speech']), orig_sr=actual_sr, target_sr=target_sr)
            now = datetime.now()
            # dd/mm/YY H:M:S
            dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
            print('Resampling completes', dt_string)
        else:
            print('MATCH!: The sampling rate used for fine-tuning Wav2vec2.0 matches the sampling rate used for pretraining')    

In [26]:
print('\n------------------Ckech sampling rates of training datasets... ------------------\n')
check_sampling_rate(dataset['train'])


------------------Ckech sampling rates of training datasets... ------------------

MATCH!: The sampling rate used for fine-tuning Wav2vec2.0 matches the sampling rate used for pretraining


In [27]:
print('\n------------------Ckeck sampling rates of development datasets... ------------------\n')
check_sampling_rate(dataset['dev'])


------------------Ckeck sampling rates of development datasets... ------------------

MATCH!: The sampling rate used for fine-tuning Wav2vec2.0 matches the sampling rate used for pretraining


In [28]:
print('\n------------------Ckech sampling rates of testing datasets... ------------------\n')
check_sampling_rate(dataset['test'])


------------------Ckech sampling rates of testing datasets... ------------------

MATCH!: The sampling rate used for fine-tuning Wav2vec2.0 matches the sampling rate used for pretraining


In [29]:
#print(list(set(dataset['train']['sampling_rate']))[0])
#print(len(set(dataset['train']['sampling_rate'])))

In [32]:
print("\n------------------ Verifying some ramdom samples... ------------------\n")
# verify if the data is a 1-dimensional array, the sampling rate corresponds to 16kHz, and the target text is clean.
for i in range(6):
    rand_int = random.randint(0, len(dataset["train"]))
    #rand_int = i
    print('Target text:', dataset['train'][rand_int]['target_text'])
    print('Input array shape:', np.asarray(dataset['train'][rand_int]['speech']).shape)
    print('Sampling rate:', dataset['train'][rand_int]["sampling_rate"])
    print('\n')


------------------ Verifying some ramdom samples... ------------------

Target text: join
Input array shape: (41748,)
Sampling rate: 16000


Target text: civilized
Input array shape: (44745,)
Sampling rate: 16000


Target text: zucchini
Input array shape: (41744,)
Sampling rate: 16000


Target text: hopeful
Input array shape: (31813,)
Sampling rate: 16000


Target text: picture
Input array shape: (43835,)
Sampling rate: 16000


Target text: athlete
Input array shape: (31809,)
Sampling rate: 16000




In [33]:
dataset['test']['sampling_rate'][0]

16000

In [34]:
def prepare_dataset(batch):
    batch["input_values"] = processor(batch['speech'], sampling_rate=batch["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [35]:
print("\n------------------ Obtaining input values, input length, and labels... ------------------\n")
#prep_dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names["train"], num_proc=4)
dataset = dataset.map(prepare_dataset, num_proc=4)


------------------ Obtaining input values, input length, and labels... ------------------



Loading cached processed dataset at /srv/scratch/chacmod/.cache/huggingface/datasets/Jordan-OGI-finetune/csv/default-631b16b4329d1149/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-b47afbb9ff5efe2a_*_of_00004.arrow
Loading cached processed dataset at /srv/scratch/chacmod/.cache/huggingface/datasets/Jordan-OGI-finetune/csv/default-631b16b4329d1149/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-ffbd0277653c773f_*_of_00004.arrow
Loading cached processed dataset at /srv/scratch/chacmod/.cache/huggingface/datasets/Jordan-OGI-finetune/csv/default-631b16b4329d1149/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-aa0464f6546e614b_*_of_00004.arrow


In [39]:
dataset['train'][1]['labels']

[12, 10, 8, 6, 13, 13, 6, 10, 0]

In [36]:
#print(prep_dataset)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['speech', 'sampling_rate', 'target_text', 'input_values', 'input_length', 'labels'],
        num_rows: 470
    })
    dev: Dataset({
        features: ['speech', 'sampling_rate', 'target_text', 'input_values', 'input_length', 'labels'],
        num_rows: 134
    })
    test: Dataset({
        features: ['speech', 'sampling_rate', 'target_text', 'input_values', 'input_length', 'labels'],
        num_rows: 132
    })
})


In [41]:
rannn=(torch.tensor(dataset['test'][0]['input_values']).unsqueeze(0))
print(rannn.size())
print(type(rannn))


torch.Size([1, 82210])
<class 'torch.Tensor'>


In [63]:
#---------------Training + Evaluation--------------------
# ** Define a padding data collator used to pad the training samples to longest sample in their batch. In contrast to most NLP models,
#    Wav2Vec2 has a much larger input length than output length. E.g., a sample of input length 50000 has an output length of no more than 100. 
#    Given the large input sizes, it is much more efficient to pad the training batches dynamically meaning that all training samples should only 
#    be padded to the longest sample in their batch and not the overall longest sample. Therefore, fine-tuning Wav2Vec2 requires a special padding data collator, 
#    which we will define below
# ** Evaluation metric. During training, the model should be evaluated on the word error rate. We should define a compute_metrics function accordingly
# ** Load a pretrained checkpoint. We need to load a pretrained checkpoint and configure it correctly for training.
# ** Define the training configuration.

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    
    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace label ids of padding tokens with -100 so that those tokens are not taken into account when computing the loss
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [64]:
print('\n------------------ Setting up the padding data collator... ------------------\n')
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)


------------------ Setting up the padding data collator... ------------------



In [65]:
print('\n------------------ Setting up WER metric... ------------------\n')
wer_metric = load_metric("wer")


------------------ Setting up WER metric... ------------------



/scratch/pbs.4453745.kman.restech.unsw.edu.au/ipykernel_2452574/3043604959.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


In [69]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [70]:
print('\n------------------ Loading a pretrained checkpount... ------------------\n')
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base", 
    pad_token_id=processor.tokenizer.pad_token_id,
    hidden_dropout=set_hidden_dropout,
    activation_dropout=set_activation_dropout,
    attention_dropout=set_attention_dropout,
    feat_proj_dropout=set_feat_proj_dropout,
    layerdrop=set_layerdrop,
    mask_time_prob=set_mask_time_prob,
    mask_time_length=set_mask_time_length,
    ctc_loss_reduction=set_ctc_loss_reduction,
    ctc_zero_infinity=set_ctc_zero_infinity    
)


------------------ Loading a pretrained checkpount... ------------------



Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForCTC: ['project_q.weight', 'quantizer.codevectors', 'quantizer.weight_proj.weight', 'project_q.bias', 'quantizer.weight_proj.bias', 'project_hid.weight', 'project_hid.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [71]:
# CNN layers of Wav2vec2.0 model is sufficiently trained, hence they do not need to be finetuned anymore
model.freeze_feature_encoder()

In [72]:
print('\n------------------ Setting TrainingArguments... ------------------\n')
training_args = TrainingArguments(
    output_dir=model_fp,
    per_device_train_batch_size=set_per_device_train_batch_size,
    group_by_length=set_group_by_length,
    gradient_accumulation_steps=set_gradient_accumulation_steps,
    gradient_checkpointing=set_gradient_checkpointing,
    weight_decay=set_weight_decay,
    fp16=set_fp16,
    learning_rate=set_learning_rate,
    lr_scheduler_type=set_lr_scheduler_type,
    adam_beta1=set_adam_beta1,
    adam_beta2=set_adam_beta2,
    adam_epsilon=set_adam_epsilon,
    warmup_ratio=set_warmup_ratio,
    num_train_epochs=set_num_train_epochs,
    max_steps=set_max_steps,
    logging_strategy=set_logging_strategy,
    logging_steps=set_logging_steps,
    save_strategy=set_save_strategy,
    save_steps=set_save_steps,
    evaluation_strategy=set_evaluation_strategy,
    eval_steps=set_eval_steps,
    save_total_limit=set_save_total_limit,
    load_best_model_at_end=set_load_best_model_at_end,
    metric_for_best_model=set_metric_for_best_model,
    greater_is_better=set_greater_is_better
)


------------------ Setting TrainingArguments... ------------------



In [73]:
print('\n------------------ Setting Trainer... ------------------\n')
    trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset["train"],
    eval_dataset=dataset["dev"],
    tokenizer=processor.feature_extractor
)


------------------ Setting Trainer... ------------------



In [ ]:
torch.cuda.empty_cache()
trainer.train()
model.save_pretrained(model_fp)

/home/z5313567/.local/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/home/z5313567/.local/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


In [ ]:
print('\n------------------ Evaluation starts... ------------------\n')
processor = Wav2Vec2Processor.from_pretrained(model_fp)


In [ ]:
model = Wav2Vec2ForCTC.from_pretrained("patrickvonplaten/wav2vec2-base-timit-demo-google-colab").cuda()

In [ ]:
def map_to_result(batch):
  model.to("cuda")
  input_values = processor(
      batch["speech"], 
      sampling_rate=batch["sampling_rate"], 
      return_tensors="pt"
  ).input_values.to("cuda")

  with torch.no_grad():
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  
  return batch

In [ ]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)
  
  return batch